# Word2Vec Model

In [2]:
import numpy as np
import pandas as pd
import re
import warnings
from gensim.models import Word2Vec
import mlflow
warnings.filterwarnings("ignore")

In [ ]:
# !mlflow server --host 127.0.0.1 --port 5000

[2025-07-17 21:02:02 -0700] [55781] [INFO] Starting gunicorn 23.0.0
[2025-07-17 21:02:02 -0700] [55781] [INFO] Listening at: http://127.0.0.1:5000 (55781)
[2025-07-17 21:02:02 -0700] [55781] [INFO] Using worker: sync
[2025-07-17 21:02:02 -0700] [55782] [INFO] Booting worker with pid: 55782
[2025-07-17 21:02:02 -0700] [55783] [INFO] Booting worker with pid: 55783
[2025-07-17 21:02:03 -0700] [55784] [INFO] Booting worker with pid: 55784
[2025-07-17 21:02:03 -0700] [55785] [INFO] Booting worker with pid: 55785
^C

Aborted!
[2025-07-17 21:02:06 -0700] [55785] [INFO] Worker exiting (pid: 55785)
[2025-07-17 21:02:06 -0700] [55783] [INFO] Worker exiting (pid: 55783)
[2025-07-17 21:02:06 -0700] [55784] [INFO] Worker exiting (pid: 55784)
[2025-07-17 21:02:06 -0700] [55782] [INFO] Worker exiting (pid: 55782)


In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("word2vec_experiment")

In [3]:
import requests
import json
import boto3
import streamlit as st

In [12]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from fastapi import FastAPI
import uvicorn
import numpy as np
import pandas as pd
import json
import boto3
import streamlit as st

def process_recipe_vector_means(data: pd.DataFrame, _model: Word2Vec) -> list:
    """Takes in a list of recipe ingredients, embeds it and calculate the mean"""
    recipe_vector_means = []

    for recipe_ingredients in data.ingredients:
        embedding_vec = [model.wv[ing] for ing in recipe_ingredients if ing in model.wv]
        mean_vec = np.mean(embedding_vec, axis=0) if embedding_vec else np.zeros(model.vector_size) * -1
        recipe_vector_means.append(mean_vec)
    
    return np.array(recipe_vector_means)

def connect_database():
    """Connect to the DynamoDB database."""
    dynamodb = boto3.resource("dynamodb",
                            aws_access_key_id=st.secrets.s3.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=st.secrets.s3.AWS_SECRET_ACCESS_KEY,
                            region_name=st.secrets.s3.AWS_DEFAULT_REGION)
    table = dynamodb.Table(st.secrets.s3.DB_NAME)
    return table

app = FastAPI(title="Food-Rec_Word2Vec_API")
data = pd.read_pickle("processed_cookbook.pkl")
model = Word2Vec.load("word2vec.model")
table = connect_database()
recipe_vector_means = process_recipe_vector_means(data, model)

def connect_database():
    """Connect to the DynamoDB database."""
    dynamodb = boto3.resource("dynamodb",
                            aws_access_key_id=st.secrets.s3.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=st.secrets.s3.AWS_SECRET_ACCESS_KEY,
                            region_name=st.secrets.s3.AWS_DEFAULT_REGION)
    table = dynamodb.Table(st.secrets.s3.DB_NAME)
    return table

table = connect_database()

user_id = 110833230122006731136
n=10

# Get user configuration from DynamoDB
user_config = table.get_item(Key={"user_id": int(user_id)})["Item"]
liked_idx = set(map(int, user_config.get("liked_idx").keys()))
disliked_idx = set(map(int, user_config.get("disliked_idx").keys()))

exclude_indices = set(liked_idx).union(disliked_idx)

# If there are no liked or disliked recipes, return random indices
if not exclude_indices:
    print(np.random.choice(len(recipe_vector_means), n, replace=False))
    
user_vec = np.zeros(recipe_vector_means.shape[1]) + np.sum(recipe_vector_means[list(liked_idx)], axis=0) - np.sum(recipe_vector_means[list(disliked_idx)], axis=0)
sims = cosine_similarity(user_vec.reshape(1, -1), recipe_vector_means)[0]

# Create a DataFrame to hold the recipe IDs and their similarity scores
recipe_similarity = pd.DataFrame({
    "id": data.id,
    "similarity": sims
})

recipe_similarity = recipe_similarity.sort_values(by="similarity", ascending=False) # Sort by similarity in descending order

# Exclude liked and disliked recipes from the similarity scores
if exclude_indices:
    recipe_similarity = recipe_similarity[~recipe_similarity["id"].isin(exclude_indices)]

arg_sorted_sims = recipe_similarity["similarity"].argsort()[::-1]  # Sort indices by similarity in descending order
# arg_pool = np.append(arg_sorted_sims[:n], arg_sorted_sims[n:(len(arg_sorted_sims)//2)])  # Get a pool of indices to choose from
rand = np.random.choice(arg_sorted_sims[:n], n, replace=False)

# np.random.choice(arg_sorted_sims[:n], n, replace=False)


In [13]:
user_id = 110833230122006731136
table_test = table.get_item(Key={"user_id": int(user_id)})["Item"]

table_test

{'disliked_idx': {'5948': '2025-07-17 22:42:40',
  '1106': '2025-07-17 22:42:25',
  '2227': '2025-07-17 22:42:37',
  '1751': '2025-07-17 22:48:32',
  '5869': '2025-07-17 22:42:36'},
 'user_id': Decimal('110833230122006731136'),
 'liked_idx': {'795': '2025-07-17 22:42:39',
  '1705': '2025-07-17 22:48:49',
  '1758': '2025-07-17 22:42:24',
  '425': '2025-07-17 22:42:38',
  '1438': '2025-07-17 22:48:30'}}

In [138]:
table_test.get("liked_idx")

{'5717': '2025-07-17'}

In [153]:
from datetime import datetime
test_map = {"liked_idx": list(map(int, list(liked_idx))), 
            "disliked_idx": list(map(int, list(disliked_idx))),
            "date": datetime.now().strftime("%Y-%m-%d" + "%H:%M:%S")}
test_map


{'liked_idx': [2128, 277, 5509],
 'disliked_idx': [2819, 195, 3719, 2378, 2092, 916],
 'date': '2025-07-1722:41:21'}

In [7]:
liked_idx = set(table_test.get("liked_idx").keys())
disliked_idx = set(table_test.get("disliked_idx").keys())

exclude_indices = set(liked_idx).union(disliked_idx)


In [8]:
exclude_indices

{'1106', '1758', '2227', '425', '5869', '5948', '795'}

In [128]:
table.update_item(
        Key={"user_id": int(user_id)},
        AttributeUpdates={
            "test_idx": {
                "Value": {int(123): datetime.datetime.now().strftime("%Y-%m-%d")},
                "Action": "PUT"
            },
        }
    )

ParamValidationError: Parameter validation failed:
Invalid type for parameter AttributeUpdates.test_idx.Value.M (key: 123), value: 123, type: <class 'int'>, valid types: <class 'str'>

In [159]:
np.delete([1, 2, 3, 4, 5], 2)  # Example of deleting specific indices

array([1, 2, 4, 5])

In [135]:
np.int64(123)

123

In [ ]:
req = requests.get("http://localhost:8000/recommend/110833230122006731136/8")
np.asarray(json.loads(req.json()))

array([3052, 2622, 3155, 2778, 3129, 1556, 1200, 3065])

In [127]:
test = {}
test[0] = 1
test[1] = 2
test[2] = 3

test.values()



dict_values([1, 2, 3])

In [ ]:
data = pd.read_pickle("processed_cookbook.pkl")

In [16]:
data[data.id.isin([5, 3, 2])]

,id,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
2,2,Homemade Udon Noodles,https://www.justonecookbook.com/udon-noodles/,"[allpurpose_flour, water, salt, potato_starch]",36.0,120.0,60.0,0.0,150.0,361.0,...,198.0,106.0,3.0,1.0,0.0,0.0,21.0,5.0,0.0,https://www.justonecookbook.com/wp-content/upl...
3,3,Tomato Egg Vermicelli Soup,https://www.justonecookbook.com/tomato-egg-ver...,"[tomato, green_scallion, egg, chicken_broth, s...",10.0,30.0,5.0,10.0,0.0,123.0,...,299.0,215.0,2.0,3.0,737.0,11.0,79.0,2.0,0.0,https://www.justonecookbook.com/wp-content/upl...
5,5,Honey Gochujang Chicken,https://www.justonecookbook.com/honey-gochujan...,"[garlic, chicken_breast, salt, black_pepper, p...",13.0,40.0,10.0,10.0,0.0,408.0,...,813.0,723.0,1.0,12.0,65.0,4.0,26.0,1.0,0.0,https://www.justonecookbook.com/wp-content/upl...


In [ ]:
# Skip-gram works better for infrequent words and lower data size
# CBOW works better for frequent words and larger data size
w2v_model = Word2Vec(data.ingredients, vector_size=100, window=11, sg=0, epochs=1000)

In [5]:
# w2v_model.save("word2vec.model")
w2v_model = Word2Vec.load("word2vec.model")


In [24]:
w2v_model.wv.most_similar("salt", topn=10)

[('salt_black_pepper', 0.22019974887371063),
 ('water', 0.21628636121749878),
 ('red_chili', 0.2155621200799942),
 ('snow_pea', 0.20578241348266602),
 ('onion', 0.20268560945987701),
 ('spicy_bean', 0.2002008855342865),
 ('carrot', 0.1976989507675171),
 ('salt_plus', 0.18816617131233215),
 ('garlic', 0.17746944725513458),
 ('worcestershire', 0.17670156061649323)]

In [17]:
# Choose median length of ingredients as window size
ingredient_length = np.array([len(ingredient) for ingredient in data.ingredients])
np.ceil(np.median(ingredient_length))

11.0

In [18]:
w2v_model.wv.most_similar("oil")

[('vegetable_oil', 0.6726031303405762),
 ('neutral_oil', 0.5508599281311035),
 ('peanut_oil', 0.4571745693683624),
 ('olive_oil', 0.4171428978443146),
 ('canola_oil', 0.3822813630104065),
 ('lowsodium_chicken_stock', 0.30548906326293945),
 ('chinese_wine', 0.2943076491355896),
 ('chili_oil', 0.27140793204307556),
 ('scallion', 0.261827290058136),
 ('tomato_passata', 0.26169365644454956)]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_recipe_vector_mean(ingredients: list, model: Word2Vec) -> list:
    """
    Takes in a list of recipe ingredients, embeds it and calculate the mean
    """
    vector_embedding = [model.wv[ing] for ing in ingredients if ing in model.wv]
    return np.mean(vector_embedding, axis=0) if vector_embedding else np.zeros(model.vector_size) * -1
    

In [65]:
# recipes_vector_mean = [get_recipe_vector_mean(recipe, w2v_model) for recipe in data.ingredients]
# # get_recipe_vector_mean(["blueberries", "salt", "oil", "egg", "flour", "apple"], w2v_model)
# target_recipe_mean = get_recipe_vector_mean(["salmon", "cheese", "broccoli",
#                                              "garlid_bread", "fries"], w2v_model)
# get_top_n_similarities(target_recipe_mean, recipes_vector_mean, 5)

recipes_vector_mean = [get_recipe_vector_mean(recipe, w2v_model) for recipe in data.ingredients]
recipes_vector_mean = np.array(recipes_vector_mean)

sims = cosine_similarity(recipes_vector_mean[0].reshape(1, -1), recipes_vector_mean)[0]
sims.argsort()[::-1]

array([   0,  189,  232, ..., 1285, 1588, 6148])

In [ ]:
recipes_vector_mean = [get_recipe_vector_mean(recipe, w2v_model) for recipe in data.ingredients]
recipes_vector_mean = np.array(recipes_vector_mean)

class User:
    def __init__(self, vector_dim):
        self.vec = np.zeros(vector_dim)
        self.dislike_step = 0.3
        self.like_step = 1
        self.exclude_idx = set()

    def like(self, recipe_vector_mean):
        self.vec += self.like_step * recipe_vector_mean

    def dislike(self, recipe_vector_mean):
        self.vec -= self.dislike_step * recipe_vector_mean

    def recommend(self, n):
        if not self.vec.any():
            rand_idx = np.random.choice(len(recipes_vector_mean), n, replace=False)
            return rand_idx

        sims = cosine_similarity(self.vec.reshape(1, -1), recipes_vector_mean)[0]

        if self.exclude_idx:
            sims[list(self.exclude_idx)] = -1 # Exclude recipes shown

        return sims.argsort()[::-1][:n]
    
test_user = User(w2v_model.vector_size)

In [74]:
idx = test_user.recommend(1)
print(idx)
recipes_vector_mean[0]
# print(data.recipe_url.iloc[idx[0]])
w2v_model.vector_size

[2404]


100

In [386]:
from time import sleep

user_input = 0

while True:
    
    if user_input == "-1":
        print("Exiting the recommendation system.", flush=True)
        break

    rec_idx = test_user.recommend(1)
    rec_url = data.recipe_url.iloc[rec_idx[0]]

    print(rec_url, flush=True)

    user_input = input("0 for dislike, 1 for like, -1 to quit: ")

    if user_input == "1":
        test_user.like(recipes_vector_mean[rec_idx[0]])
        print("Liked the recipe!", flush=True)
    else:
        test_user.dislike(recipes_vector_mean[rec_idx[0]])
        print("Disliked the recipe!", flush=True)

    test_user.exclude_idx.add(rec_idx[0])

https://thewoksoflife.com/homemade-chinese-egg-noodles/
Liked the recipe!
https://thewoksoflife.com/liangpi-noodles/
Liked the recipe!
https://thewoksoflife.com/nai-wong-bao-custard-buns/
Liked the recipe!
https://thewoksoflife.com/milk-bread-croissants-recipe/
Disliked the recipe!
https://thewoksoflife.com/flaky-apple-pie-recipe/
Disliked the recipe!
https://thewoksoflife.com/hong-kong-egg-tarts/
Liked the recipe!
https://natashaskitchen.com/sweet-cherry-filled-buns-vatrushka-recipe/
Liked the recipe!
https://www.recipetineats.com/brioche/
Disliked the recipe!
https://thewoksoflife.com/peach-squares/
Disliked the recipe!
https://thewoksoflife.com/lotus-leaf-buns/
Liked the recipe!
https://thewoksoflife.com/steamed-red-bean-buns/
Disliked the recipe!
https://thewoksoflife.com/chinese-sausage-buns/
Liked the recipe!
https://thewoksoflife.com/chinese-pineapple-buns-pork/
Disliked the recipe!
https://natashaskitchen.com/snickerdoodle-cookies/
Disliked the recipe!
https://natashaskitchen.c

In [ ]:
# data.iloc[test_user.recommend(100), :]

# data.iloc[cosine_similarity(test_user.vec.reshape(1, -1), recipes_vector_mean).argsort()[0][::-1], :]

['asdasd', 'asd', 'asd', 'a', 'sd', 'a', 'asdasd', 'asd']

In [88]:
np.hstack([recipes_vector_mean[0], [100, 100, 0]])

array([-1.91354714e-02,  6.63986132e-02,  8.90939236e-02, ...,
        1.00000000e+02,  1.00000000e+02,  0.00000000e+00])

In [5]:
np.mean([w2v_model.wv[k] for k in ["salt", "water"]], axis=0)

array([-0.27168563, -0.16897143,  0.3787686 , -0.09206532,  0.04078965,
       -0.06412343,  0.10212671, -0.23964828, -0.08686227, -0.20235255,
        0.29980344, -0.255956  , -0.06663971,  0.24358888,  0.19246778,
       -0.12094   , -0.0811328 , -0.02435989, -0.1146047 , -0.16485226,
       -0.03452671, -0.0789252 ,  0.1704962 ,  0.0194522 ,  0.24522936,
       -0.0017973 , -0.24600455, -0.11081199, -0.06806016, -0.0466217 ,
       -0.33850163,  0.10324778,  0.00628475, -0.31563273,  0.01918057,
        0.22137383, -0.08368203, -0.12495574,  0.12044365, -0.20289102,
        0.2751795 , -0.2860985 , -0.09349979,  0.00259076,  0.11370799,
       -0.2269685 ,  0.10482226,  0.15076938, -0.14638446, -0.21089286,
       -0.21453089, -0.02883867, -0.03782435,  0.11072452,  0.00501031,
        0.0309107 ,  0.06699391,  0.10127933,  0.07713623,  0.15280007,
        0.12606956,  0.05258124, -0.20845813, -0.24534018, -0.26487112,
        0.19250645,  0.10080535, -0.06604837, -0.03239607, -0.13